In [ ]:
import datatable as dt
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
import janestreet
from tqdm import tqdm
import pandas as pd
import os

class Model(nn.Module):
    def __init__(self, all_feat_cols, hidden_size, target_cols):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(all_feat_cols)
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(all_feat_cols, hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+all_feat_cols, hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, target_cols)

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        # self.GeLU = nn.GELU()
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        # x = F.relu(x)
        # x = self.PReLU(x)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Model(131, 256, 2)
# model = model.to(device)
model.load_state_dict(torch.load('../input/janestreetstate/best_30.pth', map_location='cpu'))

model.eval()
X_test = None
env = janestreet.make_env()
env_iter = env.iter_test()
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        test_df = test_df.loc[:, 'weight':'feature_129']
        test_df = test_df.fillna(test_df.mean())
#         if X_test is None:
#             X_test = np.concatenate([test_df for _ in range(seq_dim)],axis=0)
#         X_test = np.concatenate([X_test[1:], test_df] ,axis=0)
#         preds = model(torch.tensor(X_test[np.newaxis,:], dtype=torch.float).to(device))
#         preds = preds.cpu().detach().numpy()
#         action = ((test_df['weight'].values * preds[:, 1]) > 0).astype('int')
        preds = model(torch.from_numpy(test_df.values).to(torch.float32))
        preds = F.log_softmax(preds, dim=1).argmax(dim=1)
        action = preds.cpu().detach().numpy()
        pred_df.action = action
    else:
        pred_df.action = 0
    env.predict(pred_df)